# 05 - Walk-Forward Backtest

## Le notebook le plus important du projet

Ce notebook implémente le **backtest walk-forward** : la seule méthode honnête pour évaluer un modèle de prédiction sportive.

### Qu'est-ce que le walk-forward backtesting ?

Le walk-forward backtest simule exactement ce qui se passe en production :

1. **Entraîner** le modèle sur toutes les données disponibles **avant** la journée cible
2. **Prédire** les matchs de cette journée
3. **Comparer** aux résultats réels après le match
4. **Avancer** d'une journée et recommencer

```
Temps ──────────────────────────────────────►

Fenêtre 1: [=====TRAIN=====][PRED]  
Fenêtre 2: [======TRAIN======][PRED]  
Fenêtre 3: [=======TRAIN=======][PRED]  
...  
```

### Pourquoi c'est critique ?

- **Pas de fuite de données** : le modèle ne voit jamais le futur
- **Réaliste** : reproduit les conditions réelles de pari
- **Fiable** : mesure la vraie capacité prédictive, pas le surapprentissage
- **Décisionnel** : détermine si le modèle est prêt pour la production (GO/NO-GO)

### Métriques évaluées

| Métrique | Seuil GO | Description |
|----------|----------|-------------|
| Brier Score | < 0.22 | Précision globale des probabilités (0 = parfait, 0.25 = pile ou face) |
| ECE | < 0.08 | Erreur de calibration attendue (les 70% arrivent-ils 70% du temps ?) |
| ROI | > -2% | Retour sur investissement sur paris simulés |
| Échantillon | >= 100 paris | Taille minimale pour significativité statistique |

## 1. Configuration et imports

In [ ]:
# --- Manipulation du sys.path pour importer depuis src/ ---
import sys
import os
from pathlib import Path

# Remonter au répertoire racine du projet
PROJECT_ROOT = Path(os.getcwd()).parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Racine du projet : {PROJECT_ROOT}")
print(f"Répertoire de données : {PROJECT_ROOT / 'data' / 'raw'}")

In [ ]:
# --- Imports standards ---
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from IPython.display import display, Markdown

# --- Imports du projet ---
from src.evaluation.backtest import WalkForwardBacktest, BacktestReport, BettingResult
from src.evaluation.calibration import CalibrationReport

# --- Configuration de l'affichage ---
warnings.filterwarnings("ignore", category=FutureWarning)
sns.set_theme(style="whitegrid", palette="muted", font_scale=1.1)
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["figure.dpi"] = 100
pd.set_option("display.max_columns", 30)
pd.set_option("display.float_format", "{:.4f}".format)

print("Imports chargés avec succès.")

## 2. Chargement des données de matchs

In [ ]:
# --- Chargement des fichiers CSV depuis data/raw/ ---
RAW_DIR = PROJECT_ROOT / "data" / "raw"

csv_files = sorted(RAW_DIR.glob("*.csv"))
print(f"Fichiers CSV trouvés dans {RAW_DIR} : {len(csv_files)}")
for f in csv_files:
    print(f"  - {f.name}")

if not csv_files:
    print("\nAucun fichier CSV trouvé. Génération de données synthétiques pour démonstration...")
    print("En production, placez vos fichiers CSV dans data/raw/ avec les colonnes :")
    print("  home_team, away_team, home_score, away_score, kickoff, matchday")
    print("  + optionnel : home_odds, draw_odds, away_odds")

In [ ]:
def load_match_data(csv_files: list[Path]) -> pd.DataFrame:
    """Charger et consolider tous les CSV de matchs.
    
    Colonnes attendues : home_team, away_team, home_score, away_score, kickoff
    Colonnes optionnelles : matchday, home_odds, draw_odds, away_odds
    """
    if not csv_files:
        return pd.DataFrame()
    
    frames = []
    for f in csv_files:
        try:
            df = pd.read_csv(f)
            # Vérifier les colonnes obligatoires
            required = {"home_team", "away_team", "home_score", "away_score", "kickoff"}
            if not required.issubset(set(df.columns)):
                print(f"  ATTENTION: {f.name} manque des colonnes requises ({required - set(df.columns)})")
                continue
            df["source_file"] = f.name
            frames.append(df)
            print(f"  Chargé {f.name} : {len(df)} matchs")
        except Exception as e:
            print(f"  ERREUR lors du chargement de {f.name} : {e}")
    
    if not frames:
        return pd.DataFrame()
    
    # Concaténation et nettoyage
    df = pd.concat(frames, ignore_index=True)
    df["kickoff"] = pd.to_datetime(df["kickoff"], utc=True)
    df = df.sort_values("kickoff").reset_index(drop=True)
    
    # Supprimer les doublons potentiels
    df = df.drop_duplicates(subset=["home_team", "away_team", "kickoff"], keep="first")
    
    return df


def generate_synthetic_data(n_matches: int = 400) -> pd.DataFrame:
    """Générer des données synthétiques pour démonstration.
    
    Simule une saison complète de Ligue 1 (20 équipes, 380 matchs).
    Les données synthétiques permettent de tester le pipeline sans API.
    """
    np.random.seed(42)
    
    equipes = [
        "Paris Saint-Germain", "Olympique de Marseille", "AS Monaco",
        "Olympique Lyonnais", "LOSC Lille", "OGC Nice", "RC Lens",
        "Stade Rennais", "Montpellier HSC", "Toulouse FC",
        "Stade de Reims", "FC Nantes", "RC Strasbourg", "Le Havre AC",
        "FC Metz", "Clermont Foot", "FC Lorient", "Stade Brestois",
        "Angers SCO", "AJ Auxerre"
    ]
    
    # Forces relatives des équipes (attaque / défense)
    forces = {
        equipes[0]: (2.2, 0.7), equipes[1]: (1.5, 0.9), equipes[2]: (1.6, 0.85),
        equipes[3]: (1.4, 0.95), equipes[4]: (1.3, 0.85), equipes[5]: (1.2, 0.9),
        equipes[6]: (1.1, 0.85), equipes[7]: (1.1, 0.95), equipes[8]: (1.0, 1.0),
        equipes[9]: (0.95, 1.0), equipes[10]: (0.9, 0.9), equipes[11]: (0.9, 1.05),
        equipes[12]: (0.85, 1.0), equipes[13]: (0.8, 1.1), equipes[14]: (0.8, 1.15),
        equipes[15]: (0.75, 1.2), equipes[16]: (0.75, 1.15), equipes[17]: (0.9, 0.95),
        equipes[18]: (0.7, 1.2), equipes[19]: (0.7, 1.15),
    }
    
    matchs = []
    date_debut = datetime(2024, 8, 10)
    journee = 0
    
    for i, home in enumerate(equipes):
        for j, away in enumerate(equipes):
            if i == j:
                continue
            if len(matchs) >= n_matches:
                break
            
            att_h, def_h = forces[home]
            att_a, def_a = forces[away]
            
            # Simulation Poisson avec avantage domicile
            lambda_h = 1.35 * att_h * def_a * 1.25
            lambda_a = 1.35 * att_a * def_h
            
            home_score = np.random.poisson(lambda_h)
            away_score = np.random.poisson(lambda_a)
            
            journee = len(matchs) // 10 + 1
            date_match = date_debut + pd.Timedelta(days=journee * 7 + np.random.randint(0, 3))
            
            # Générer des cotes simulées (avec marge bookmaker ~5%)
            p_home = max(0.10, min(0.85, lambda_h / (lambda_h + lambda_a) * 0.55 + 0.15))
            p_draw = max(0.15, 0.28 - abs(p_home - 0.5) * 0.3)
            p_away = 1.0 - p_home - p_draw
            
            marge = 1.05  # 5% de marge bookmaker
            home_odds = round(marge / max(p_home, 0.05), 2)
            draw_odds = round(marge / max(p_draw, 0.05), 2)
            away_odds = round(marge / max(p_away, 0.05), 2)
            
            matchs.append({
                "home_team": home,
                "away_team": away,
                "home_score": int(home_score),
                "away_score": int(away_score),
                "kickoff": date_match.isoformat() + "Z",
                "matchday": journee,
                "home_odds": home_odds,
                "draw_odds": draw_odds,
                "away_odds": away_odds,
            })
        if len(matchs) >= n_matches:
            break
    
    df = pd.DataFrame(matchs)
    df["kickoff"] = pd.to_datetime(df["kickoff"], utc=True)
    df = df.sort_values("kickoff").reset_index(drop=True)
    return df

In [ ]:
# --- Charger les données réelles ou générer des données synthétiques ---
df_matches = load_match_data(csv_files)

if df_matches.empty:
    print("Utilisation des données synthétiques pour démonstration.")
    df_matches = generate_synthetic_data(n_matches=400)
    DATA_SOURCE = "Synthétique (démonstration)"
else:
    DATA_SOURCE = "Réel (data/raw/)"

print(f"\nSource des données : {DATA_SOURCE}")
print(f"Nombre total de matchs : {len(df_matches)}")
print(f"Période : {df_matches['kickoff'].min()} → {df_matches['kickoff'].max()}")
print(f"Équipes uniques : {df_matches['home_team'].nunique()}")
print(f"\nColonnes disponibles : {list(df_matches.columns)}")

display(df_matches.head(10))

In [ ]:
# --- Préparer les données au format attendu par WalkForwardBacktest ---

# Convertir le DataFrame en liste de dicts pour le backtest
all_matches = df_matches.to_dict(orient="records")

# Préparer le dictionnaire de cotes (si disponibles)
odds_data = None
has_odds = all(
    col in df_matches.columns
    for col in ["home_odds", "draw_odds", "away_odds"]
)

if has_odds:
    odds_data = {}
    for _, row in df_matches.iterrows():
        match_key = f"{row['home_team']}_vs_{row['away_team']}"
        odds_data[match_key] = {
            "home_odds": row["home_odds"],
            "draw_odds": row["draw_odds"],
            "away_odds": row["away_odds"],
        }
    print(f"Cotes disponibles pour {len(odds_data)} matchs → calcul des edges activé")
else:
    print("Pas de cotes disponibles → backtest sans simulation de paris")
    print("Colonnes de cotes manquantes. Ajoutez home_odds, draw_odds, away_odds aux CSV.")

print(f"\nMatchs prêts pour le backtest : {len(all_matches)}")

## 3. Exécution du Walk-Forward Backtest

In [ ]:
# --- Configuration du backtest ---
# min_training_matches : nombre minimum de matchs avant la première prédiction
# min_edge_pct : seuil minimum d'edge pour simuler un pari (en %)
# dc_weight / elo_weight : pondération de l'ensemble Dixon-Coles vs ELO

backtest = WalkForwardBacktest(
    min_training_matches=100,
    min_edge_pct=5.0,
    dc_weight=0.65,
    elo_weight=0.35,
)

print("Configuration du backtest :")
print(f"  Matchs d'entraînement minimum : {backtest.min_training}")
print(f"  Edge minimum pour pari : {backtest.min_edge}%")
print(f"  Poids Dixon-Coles : {backtest.dc_weight}")
print(f"  Poids ELO : {backtest.elo_weight}")
print(f"  Matchs à prédire : {len(all_matches) - backtest.min_training}")

In [ ]:
%%time
# --- Lancement du backtest walk-forward ---
# C'est l'étape la plus longue : chaque match est prédit après
# entraînement sur tous les matchs précédents

print("Démarrage du backtest walk-forward...")
print("(Cela peut prendre plusieurs minutes selon le nombre de matchs)\n")

report: BacktestReport = backtest.run(
    all_matches=all_matches,
    odds_data=odds_data,
)

print("\nBacktest terminé !")

## 4. Résultats du backtest

In [ ]:
# --- Affichage du rapport complet ---
print(report.summary())

In [ ]:
# --- Tableau détaillé des métriques de calibration ---
cal = report.calibration

metriques = pd.DataFrame([
    {"Métrique": "Brier Score", "Valeur": f"{cal.brier_score:.4f}", "Seuil GO": "< 0.22", "Statut": "PASS" if cal.brier_score < 0.22 else "FAIL"},
    {"Métrique": "Log Loss", "Valeur": f"{cal.log_loss:.4f}", "Seuil GO": "-", "Statut": "-"},
    {"Métrique": "ECE", "Valeur": f"{cal.ece:.4f}", "Seuil GO": "< 0.08", "Statut": "PASS" if cal.ece < 0.08 else "FAIL"},
    {"Métrique": "Accuracy", "Valeur": f"{cal.accuracy:.1%}", "Seuil GO": "-", "Statut": "-"},
    {"Métrique": "N prédictions", "Valeur": f"{cal.n_predictions}", "Seuil GO": "-", "Statut": "-"},
])

print("=" * 60)
print("MÉTRIQUES DE CALIBRATION")
print("=" * 60)
display(metriques.style.hide(axis="index"))

In [ ]:
# --- Tableau détaillé des résultats de paris ---
paris_metriques = pd.DataFrame([
    {"Métrique": "Edges trouvés (>5%)", "Valeur": f"{report.total_edges_found}"},
    {"Métrique": "Paris simulés", "Valeur": f"{len(report.betting_results)}"},
    {"Métrique": "Taux de réussite", "Valeur": f"{report.win_rate:.1%}"},
    {"Métrique": "Edge moyen", "Valeur": f"{report.avg_edge:.1f}%"},
    {"Métrique": "ROI (mise fixe)", "Valeur": f"{report.roi:.2%}"},
    {"Métrique": "PnL total", "Valeur": f"{sum(b.pnl for b in report.betting_results):.2f} unités"},
])

print("=" * 60)
print("RÉSULTATS DES PARIS SIMULÉS")
print("=" * 60)
display(paris_metriques.style.hide(axis="index"))

In [ ]:
# --- Détail des paris par marché ---
if report.betting_results:
    df_bets = pd.DataFrame([
        {
            "date": b.match_date,
            "match": f"{b.home_team} vs {b.away_team}",
            "marché": b.market,
            "prob_modèle": b.model_prob,
            "prob_bookmaker": b.fair_bookmaker_prob,
            "edge_%": b.edge_pct,
            "cote": b.best_odds,
            "résultat": b.actual_outcome,
            "gagné": b.won,
            "pnl": b.pnl,
        }
        for b in report.betting_results
    ])
    df_bets = df_bets.sort_values("date").reset_index(drop=True)
    
    # Résumé par marché
    print("\nRésumé par type de marché :")
    resume_marche = df_bets.groupby("marché").agg(
        n_paris=("pnl", "count"),
        taux_reussite=("gagné", "mean"),
        edge_moyen=("edge_%", "mean"),
        pnl_total=("pnl", "sum"),
        roi=("pnl", "mean"),
    ).round(4)
    display(resume_marche)
    
    print(f"\nDerniers 10 paris simulés :")
    display(df_bets.tail(10))
else:
    print("Aucun pari simulé (pas de cotes disponibles ou aucun edge détecté).")
    df_bets = pd.DataFrame()

## 5. Diagramme de calibration

In [ ]:
# --- Diagramme de calibration : probabilité prédite vs fréquence réelle ---
# Un modèle parfaitement calibré suit la diagonale y = x

bins = report.calibration.calibration_bins

# Filtrer les bins avec des observations
bins_non_vides = [b for b in bins if b["count"] > 0]

if bins_non_vides:
    avg_pred = [b["avg_predicted"] for b in bins_non_vides]
    avg_actual = [b["avg_actual"] for b in bins_non_vides]
    counts = [b["count"] for b in bins_non_vides]
    gaps = [b["gap"] for b in bins_non_vides]

    fig, axes = plt.subplots(1, 2, figsize=(16, 7))

    # --- Graphique 1 : Calibration ---
    ax1 = axes[0]
    
    # Diagonale parfaite
    ax1.plot([0, 1], [0, 1], "k--", linewidth=1.5, alpha=0.6, label="Calibration parfaite")
    
    # Points de calibration (taille proportionnelle au nombre d'observations)
    sizes = np.array(counts)
    sizes_normalized = 50 + 300 * (sizes / max(sizes))  # Normaliser la taille des points
    
    scatter = ax1.scatter(
        avg_pred, avg_actual,
        s=sizes_normalized,
        c=gaps,
        cmap="RdYlGn_r",
        edgecolors="black",
        linewidth=0.8,
        alpha=0.85,
        zorder=5,
    )
    
    # Ligne reliant les points
    ax1.plot(avg_pred, avg_actual, "-", color="#2196F3", linewidth=1.5, alpha=0.5, zorder=4)
    
    # Zone acceptable (ECE < 0.08)
    x_fill = np.linspace(0, 1, 100)
    ax1.fill_between(x_fill, x_fill - 0.08, x_fill + 0.08,
                     alpha=0.1, color="green", label="Zone acceptable (ECE < 0.08)")
    
    cbar = plt.colorbar(scatter, ax=ax1, shrink=0.8)
    cbar.set_label("Écart (gap)", fontsize=10)
    
    ax1.set_xlabel("Probabilité prédite", fontsize=12)
    ax1.set_ylabel("Fréquence réelle", fontsize=12)
    ax1.set_title(
        f"Diagramme de Calibration\n"
        f"ECE = {report.calibration.ece:.4f} | Brier = {report.calibration.brier_score:.4f}",
        fontsize=13, fontweight="bold",
    )
    ax1.set_xlim(-0.02, 1.02)
    ax1.set_ylim(-0.02, 1.02)
    ax1.set_aspect("equal")
    ax1.legend(loc="upper left", fontsize=9)

    # --- Graphique 2 : Histogramme des observations par bin ---
    ax2 = axes[1]
    
    bin_centers = [(b["bin_start"] + b["bin_end"]) / 2 for b in bins]
    bin_width = bins[0]["bin_end"] - bins[0]["bin_start"]
    all_counts = [b["count"] for b in bins]
    
    colors = ["#4CAF50" if b["gap"] < 0.05 else "#FF9800" if b["gap"] < 0.10 else "#F44336"
              for b in bins]
    
    ax2.bar(bin_centers, all_counts, width=bin_width * 0.85,
            color=colors, edgecolor="black", linewidth=0.5, alpha=0.8)
    
    ax2.set_xlabel("Probabilité prédite (bin)", fontsize=12)
    ax2.set_ylabel("Nombre d'observations", fontsize=12)
    ax2.set_title("Distribution des prédictions par bin", fontsize=13, fontweight="bold")
    
    # Légende manuelle pour les couleurs
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor="#4CAF50", edgecolor="black", label="Gap < 5%"),
        Patch(facecolor="#FF9800", edgecolor="black", label="Gap 5-10%"),
        Patch(facecolor="#F44336", edgecolor="black", label="Gap > 10%"),
    ]
    ax2.legend(handles=legend_elements, loc="upper right", fontsize=9)

    plt.tight_layout()
    plt.savefig(str(PROJECT_ROOT / "data" / "results" / "calibration_diagram.png"),
                dpi=150, bbox_inches="tight")
    plt.show()
    print("Diagramme sauvegardé dans data/results/calibration_diagram.png")
else:
    print("Pas assez de données de calibration pour tracer le diagramme.")

## 6. PnL cumulé au fil du temps

In [ ]:
# --- Graphique du PnL cumulé sur la durée du backtest ---
# Ce graphique montre l'évolution du profit/perte cumulé(e)
# Une courbe montante = modèle profitable

if not df_bets.empty and len(df_bets) > 0:
    df_pnl = df_bets.sort_values("date").copy()
    df_pnl["pnl_cumulé"] = df_pnl["pnl"].cumsum()
    df_pnl["pari_n"] = range(1, len(df_pnl) + 1)
    
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Remplissage vert/rouge selon le signe du PnL
    ax.fill_between(
        df_pnl["pari_n"], df_pnl["pnl_cumulé"], 0,
        where=df_pnl["pnl_cumulé"] >= 0,
        color="#4CAF50", alpha=0.15, interpolate=True,
    )
    ax.fill_between(
        df_pnl["pari_n"], df_pnl["pnl_cumulé"], 0,
        where=df_pnl["pnl_cumulé"] < 0,
        color="#F44336", alpha=0.15, interpolate=True,
    )
    
    # Courbe principale
    ax.plot(
        df_pnl["pari_n"], df_pnl["pnl_cumulé"],
        color="#1565C0", linewidth=2.0, zorder=5,
    )
    
    # Ligne de zéro
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="-", alpha=0.5)
    
    # Annotations clés
    pnl_final = df_pnl["pnl_cumulé"].iloc[-1]
    pnl_max = df_pnl["pnl_cumulé"].max()
    pnl_min = df_pnl["pnl_cumulé"].min()
    drawdown_max = (df_pnl["pnl_cumulé"].cummax() - df_pnl["pnl_cumulé"]).max()
    
    # Point final
    couleur_final = "#4CAF50" if pnl_final >= 0 else "#F44336"
    ax.scatter([len(df_pnl)], [pnl_final], color=couleur_final, s=100, zorder=10,
               edgecolors="black", linewidth=1)
    ax.annotate(
        f"PnL final : {pnl_final:+.2f}u",
        xy=(len(df_pnl), pnl_final),
        xytext=(15, 15), textcoords="offset points",
        fontsize=11, fontweight="bold", color=couleur_final,
        arrowprops=dict(arrowstyle="->", color=couleur_final, lw=1.5),
    )
    
    # Informations dans un encadré
    textstr = (
        f"ROI : {report.roi:.2%}\n"
        f"Win rate : {report.win_rate:.1%}\n"
        f"Max drawdown : {drawdown_max:.2f}u\n"
        f"N paris : {len(df_bets)}"
    )
    props = dict(boxstyle="round,pad=0.5", facecolor="wheat", alpha=0.8)
    ax.text(0.02, 0.97, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment="top", bbox=props)
    
    ax.set_xlabel("Numéro du pari", fontsize=12)
    ax.set_ylabel("PnL cumulé (unités)", fontsize=12)
    ax.set_title(
        f"Profit & Loss Cumulé - Walk-Forward Backtest\n"
        f"Mise fixe 1 unité | Edge minimum {backtest.min_edge}%",
        fontsize=13, fontweight="bold",
    )
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter("%.1f"))
    
    plt.tight_layout()
    plt.savefig(str(PROJECT_ROOT / "data" / "results" / "cumulative_pnl.png"),
                dpi=150, bbox_inches="tight")
    plt.show()
    print("Graphique sauvegardé dans data/results/cumulative_pnl.png")
else:
    print("Pas de paris simulés disponibles pour tracer le PnL cumulé.")
    print("Assurez-vous que les données contiennent des cotes (home_odds, draw_odds, away_odds).")

## 7. Distribution des edges

In [ ]:
# --- Histogramme de la distribution des edges détectés ---
# L'edge = (prob_modèle - prob_bookmaker) / prob_bookmaker * 100
# Un edge positif signifie que notre modèle pense que l'événement
# est plus probable que ce que le bookmaker propose

if not df_bets.empty and len(df_bets) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    edges = df_bets["edge_%"].values
    edges_gagnants = df_bets[df_bets["gagné"]]["edge_%"].values
    edges_perdants = df_bets[~df_bets["gagné"]]["edge_%"].values
    
    # --- Graphique 1 : Distribution globale des edges ---
    ax1 = axes[0]
    
    ax1.hist(edges, bins=25, color="#2196F3", edgecolor="black",
             linewidth=0.5, alpha=0.7, label="Tous les paris")
    
    ax1.axvline(x=np.mean(edges), color="#F44336", linewidth=2, linestyle="--",
                label=f"Moyenne : {np.mean(edges):.1f}%")
    ax1.axvline(x=np.median(edges), color="#FF9800", linewidth=2, linestyle="-.",
                label=f"Médiane : {np.median(edges):.1f}%")
    
    ax1.set_xlabel("Edge (%)", fontsize=12)
    ax1.set_ylabel("Nombre de paris", fontsize=12)
    ax1.set_title("Distribution des Edges Détectés", fontsize=13, fontweight="bold")
    ax1.legend(fontsize=9)
    
    # --- Graphique 2 : Edges gagnants vs perdants ---
    ax2 = axes[1]
    
    bins_range = np.linspace(edges.min(), edges.max(), 20)
    
    if len(edges_gagnants) > 0:
        ax2.hist(edges_gagnants, bins=bins_range, color="#4CAF50", edgecolor="black",
                 linewidth=0.5, alpha=0.6, label=f"Gagnants (n={len(edges_gagnants)})")
    if len(edges_perdants) > 0:
        ax2.hist(edges_perdants, bins=bins_range, color="#F44336", edgecolor="black",
                 linewidth=0.5, alpha=0.6, label=f"Perdants (n={len(edges_perdants)})")
    
    ax2.set_xlabel("Edge (%)", fontsize=12)
    ax2.set_ylabel("Nombre de paris", fontsize=12)
    ax2.set_title("Edges : Gagnants vs Perdants", fontsize=13, fontweight="bold")
    ax2.legend(fontsize=9)
    
    plt.tight_layout()
    plt.savefig(str(PROJECT_ROOT / "data" / "results" / "edge_distribution.png"),
                dpi=150, bbox_inches="tight")
    plt.show()
    print("Graphique sauvegardé dans data/results/edge_distribution.png")
    
    # Statistiques complémentaires
    print(f"\nStatistiques des edges :")
    print(f"  Min    : {edges.min():.1f}%")
    print(f"  Max    : {edges.max():.1f}%")
    print(f"  Moyenne: {edges.mean():.1f}%")
    print(f"  Médiane: {np.median(edges):.1f}%")
    print(f"  Écart-type: {edges.std():.1f}%")
else:
    print("Pas de paris simulés pour tracer la distribution des edges.")

## 8. Décision GO / NO-GO

In [ ]:
# --- Évaluation GO / NO-GO pour mise en production ---
# Critères stricts pour décider si le modèle est prêt

# Vérification de chaque critère
criteres = [
    {
        "critère": "Brier Score < 0.22",
        "valeur": f"{report.calibration.brier_score:.4f}",
        "seuil": 0.22,
        "ok": report.calibration.brier_score < 0.22,
        "description": "Précision globale des probabilités",
    },
    {
        "critère": "ECE < 0.08",
        "valeur": f"{report.calibration.ece:.4f}",
        "seuil": 0.08,
        "ok": report.calibration.ece < 0.08,
        "description": "Erreur de calibration attendue",
    },
    {
        "critère": "ROI > -2%",
        "valeur": f"{report.roi:.2%}",
        "seuil": -0.02,
        "ok": report.roi > -0.02,
        "description": "Retour sur investissement",
    },
    {
        "critère": "Échantillon >= 100 paris",
        "valeur": f"{len(report.betting_results)} paris",
        "seuil": 100,
        "ok": len(report.betting_results) >= 100,
        "description": "Taille minimale pour significativité",
    },
]

# Décision finale
tous_ok = all(c["ok"] for c in criteres)
decision = "GO" if tous_ok else "NO-GO"
couleur_decision = "\033[92m" if tous_ok else "\033[91m"  # Vert ou rouge ANSI
reset = "\033[0m"

print("\n" + "=" * 70)
print("  DÉCISION GO / NO-GO POUR MISE EN PRODUCTION")
print("=" * 70)

df_criteres = pd.DataFrame(criteres)
df_criteres["statut"] = df_criteres["ok"].map({True: "PASS", False: "FAIL"})
display(
    df_criteres[["critère", "valeur", "statut", "description"]]
    .style
    .hide(axis="index")
    .map(
        lambda v: "background-color: #C8E6C9" if v == "PASS" else "background-color: #FFCDD2" if v == "FAIL" else "",
        subset=["statut"],
    )
)

print(f"\n{'=' * 70}")
print(f"{couleur_decision}  >>> DÉCISION FINALE : {decision} <<<{reset}")
print(f"{'=' * 70}")

if tous_ok:
    print("\n  Le modèle satisfait tous les critères. Prêt pour un déploiement")
    print("  progressif en production avec surveillance continue.")
else:
    print("\n  Le modèle ne satisfait pas tous les critères.")
    print("  Actions recommandées :")
    for c in criteres:
        if not c["ok"]:
            print(f"    - {c['critère']} : valeur actuelle = {c['valeur']}")

In [ ]:
# --- Visualisation synthétique de la décision GO/NO-GO ---

fig, ax = plt.subplots(figsize=(10, 5))

labels = [c["critère"] for c in criteres]
statuts = [c["ok"] for c in criteres]
couleurs = ["#4CAF50" if s else "#F44336" for s in statuts]
symboles = ["PASS" if s else "FAIL" for s in statuts]

bars = ax.barh(labels, [1] * len(labels), color=couleurs, edgecolor="black",
               linewidth=0.8, alpha=0.8, height=0.6)

# Ajouter les valeurs et statuts sur les barres
for i, (bar, critere) in enumerate(zip(bars, criteres)):
    ax.text(
        0.5, bar.get_y() + bar.get_height() / 2,
        f"{critere['valeur']}  [{symboles[i]}]",
        ha="center", va="center", fontsize=12, fontweight="bold",
        color="white",
    )

# Bannière de décision
couleur_bg = "#4CAF50" if tous_ok else "#F44336"
ax.text(
    0.5, 1.08, f"DÉCISION : {decision}",
    transform=ax.transAxes, ha="center", va="center",
    fontsize=18, fontweight="bold", color="white",
    bbox=dict(boxstyle="round,pad=0.4", facecolor=couleur_bg, edgecolor="black"),
)

ax.set_xlim(0, 1)
ax.set_xticks([])
ax.set_title("Tableau de bord GO / NO-GO", fontsize=14, fontweight="bold", pad=30)
ax.invert_yaxis()  # Premier critère en haut

plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / "data" / "results" / "go_nogo_dashboard.png"),
            dpi=150, bbox_inches="tight")
plt.show()
print("Tableau de bord sauvegardé dans data/results/go_nogo_dashboard.png")

## 9. Résumé et prochaines étapes

### Ce que nous avons fait

1. **Chargé les données** de matchs depuis `data/raw/` (ou généré des données synthétiques)
2. **Exécuté le backtest walk-forward** : entraînement incrémental sur le passé, prédiction du futur
3. **Évalué la calibration** : Brier Score, Log Loss, ECE
4. **Simulé des paris** : identification des edges et suivi du PnL
5. **Pris une décision GO/NO-GO** basée sur des seuils objectifs

### Interprétation des résultats

- **Brier Score** : mesure la distance entre les probabilités prédites et les résultats réels. Un score < 0.22 indique que le modèle bat un pronostiqueur naïf (qui prédirait toujours les probabilités historiques).

- **ECE** : vérifie que les probabilités sont bien calibrées. Quand le modèle dit "70%", cela arrive-t-il vraiment ~70% du temps ? Un ECE < 0.08 est acceptable.

- **ROI** : le retour sur investissement sur des paris simulés à mise fixe. Un ROI > -2% signifie que les pertes sont contenues et que le modèle a un potentiel avec une meilleure stratégie de mise.

- **Taille de l'échantillon** : au minimum 100 paris pour avoir une signification statistique. En dessous, les résultats peuvent être dus au hasard.

### Prochaines étapes

| Étape | Description | Priorité |
|-------|-------------|----------|
| Données réelles | Remplacer les données synthétiques par les données football-data.org | Haute |
| Cotes historiques | Intégrer les cotes historiques pour un calcul d'edge réaliste | Haute |
| Optimisation des poids | Grid search sur dc_weight / elo_weight via cross-validation | Moyenne |
| Stratégie de mise | Tester Kelly criterion vs mise fixe vs mise proportionnelle | Moyenne |
| Ajout de features | xG, forme récente, blessures, weather pour le modèle ML | Basse |
| Monitoring live | Tableau de bord en temps réel avec dérive de calibration | Basse |

### Règle d'or

> **Ne jamais déployer un modèle qui n'a pas passé le backtest walk-forward.**  
> Les backtests "classiques" (train/test split fixe) surestiment systématiquement la performance.  
> Seul le walk-forward reproduit les conditions réelles d'utilisation.

In [ ]:
# --- Sauvegarde des résultats du backtest ---
results_dir = PROJECT_ROOT / "data" / "results"
results_dir.mkdir(parents=True, exist_ok=True)

# Sauvegarder le rapport en JSON
import json

rapport_dict = {
    "date_execution": datetime.now().isoformat(),
    "source_donnees": DATA_SOURCE,
    "config": {
        "min_training_matches": backtest.min_training,
        "min_edge_pct": backtest.min_edge,
        "dc_weight": backtest.dc_weight,
        "elo_weight": backtest.elo_weight,
    },
    "calibration": {
        "brier_score": report.calibration.brier_score,
        "log_loss": report.calibration.log_loss,
        "ece": report.calibration.ece,
        "accuracy": report.calibration.accuracy,
        "n_predictions": report.calibration.n_predictions,
        "is_acceptable": report.calibration.is_acceptable,
    },
    "paris": {
        "total_edges": report.total_edges_found,
        "n_paris": len(report.betting_results),
        "win_rate": report.win_rate,
        "avg_edge": report.avg_edge,
        "roi": report.roi,
        "pnl_total": sum(b.pnl for b in report.betting_results),
    },
    "decision": decision,
    "criteres": [{k: v for k, v in c.items() if k != "seuil"} for c in criteres],
}

rapport_path = results_dir / "backtest_report.json"
with open(rapport_path, "w", encoding="utf-8") as f:
    json.dump(rapport_dict, f, indent=2, ensure_ascii=False, default=str)

print(f"Rapport sauvegardé : {rapport_path}")

# Sauvegarder les paris détaillés en CSV
if not df_bets.empty:
    bets_path = results_dir / "backtest_bets.csv"
    df_bets.to_csv(bets_path, index=False)
    print(f"Détail des paris sauvegardé : {bets_path}")

print(f"\nDécision finale : {decision}")
print("Backtest terminé.")